In [1]:
import pandas as pd
import numpy as np
import os
import re
from PIL import Image
import h5py
from sklearn.model_selection import train_test_split
import argparse

In [2]:
import copy
import logging
import torch
import torch.nn as nn
from torchvision import *
import random
import sys
import time
import cv2

In [3]:
img_size = 512

img_dir =  r'ISIC2018_Task1-2_Training_Input/'
mask_dir = 'ISIC2018_Task2_Training_GroundTruth_v3/'

IMG_FILE_REGEX = r'ISIC_(\d+).jpg'
MASK_FILE_REGEX = r'ISIC_(\d+)_attribute_(.*).png'

In [4]:
Back = '../../../../../594A/Dataset/ISIC challenge 2018 Task 1-2/'

img_dir =  Back + img_dir
mask_dir = Back + mask_dir

#  ISIC2018 Val

In [5]:
df_ISIC2018 = pd.DataFrame(columns=['image', 'globules', 'milia_like_cyst', 
                                    'negative_network', 'pigment_network', 'streaks'])

In [6]:
df_ISIC2018.head()

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks


In [7]:
for img_file in os.listdir( img_dir ):
    if not img_file.endswith('.jpg'): continue
    img_id = str( re.search( IMG_FILE_REGEX, img_file ).group(1) )
    img_name = 'ISIC_' + img_id
    df_append = pd.DataFrame.from_records([{'image': img_name, 'globules': 2, 'milia_like_cyst': 2, 
                                      'negative_network': 2, 'pigment_network': 2, 'streaks': 2, 'pigment_network_pred': 2}])
    
    df_ISIC2018 = pd.concat([df_ISIC2018, df_append], ignore_index=True)

In [8]:
df_ISIC2018.head()

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks,pigment_network_pred
0,ISIC_0008913,2,2,2,2,2,2.0
1,ISIC_0000418,2,2,2,2,2,2.0
2,ISIC_0015614,2,2,2,2,2,2.0
3,ISIC_0000079,2,2,2,2,2,2.0
4,ISIC_0013399,2,2,2,2,2,2.0


## 2. save binary input 

In [9]:
'''
for mask_file in os.listdir( mask_dir ):
    if not mask_file.endswith('.png'): continue
    mask = Image.open( os.path.join( mask_dir, mask_file ) )
    mask = mask.resize( (img_size, img_size) )
    mask = np.array( mask )
    
    assert mask.shape == (img_size, img_size)
    assert mask.max() <= 255
    
    # decide feature exist or not
    if mask.max() > 0:
        binary_label = 1
    else:
        binary_label = 0
    
    m = re.search(MASK_FILE_REGEX, mask_file)
    
    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]
    
    if m.group(2) ==  'globules':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label
    elif m.group(2) ==  'milia_like_cyst':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label
    elif m.group(2) ==  'negative_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = binary_label
    elif m.group(2) ==  'pigment_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('pigment_network')]  = binary_label
    elif m.group(2) ==  'streaks':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('streaks')]  = binary_label
'''

"\nfor mask_file in os.listdir( mask_dir ):\n    if not mask_file.endswith('.png'): continue\n    mask = Image.open( os.path.join( mask_dir, mask_file ) )\n    mask = mask.resize( (img_size, img_size) )\n    mask = np.array( mask )\n    \n    assert mask.shape == (img_size, img_size)\n    assert mask.max() <= 255\n    \n    # decide feature exist or not\n    if mask.max() > 0:\n        binary_label = 1\n    else:\n        binary_label = 0\n    \n    m = re.search(MASK_FILE_REGEX, mask_file)\n    \n    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]\n    \n    if m.group(2) ==  'globules':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label\n    elif m.group(2) ==  'milia_like_cyst':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label\n    elif m.group(2) ==  'negative_network':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = bina

In [10]:
#df_ISIC2018.to_csv('df_ISIC2018_val.csv', index=False)
csv_path = '../../../../../Explainable-NN/Data/train_fold_1.csv'
df_ISIC2018 = pd.read_csv(csv_path)

In [11]:
df = df_ISIC2018.loc[(df_ISIC2018.streaks == 1)]

In [12]:
len(df)

85

# Train.PT

In [13]:
images_path = []
Masks_path = []

for image_file in df['image']:
    image_file_name = image_file + '.jpg'
    images_path.append(img_dir + image_file_name)

In [14]:
#**************Transformer**************
my_transformer = transforms.Compose([transforms.Resize((img_size,img_size)),transforms.ToTensor(),
                                 transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)) ])
#**************End   Transformer**************

In [15]:
images = torch.zeros((len(images_path), 3, 512,512), dtype = torch.float64)
for i in range(len(images_path)):
    img_path = images_path[i]
    img0 = Image.open(img_path) 
    img1 = my_transformer(img0)
    img2 = torch.tensor(img1).float().cuda() # torch.Size([3, 512, 512])
    img3 = img2.unsqueeze(0) # torch.Size([1, 3, 512, 512])

    images[i] = img3


torch.save(images, 'streaks_images.pt')

/nas/home/ruitongs/.conda/envs/asdf/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
